# Risk Aggregation POC - PLTs

This notebook reads can be used to read in multiple PLTs, rollup the PLTs to an aggregate PLT, and generate statistics and EP metrics from the PLTs.

-----

*Install necessary libraries*

In [1]:
# %pip install boto3
# %pip install fsspec
# %pip install s3fs
# %pip install pyarrow
# %pip install fastparquet

In [2]:
# Import necessary libraries
import os
import sys
sys.path.append(os.path.abspath("plttools/plt.py"))
import pandas as pd
import numpy as np
import plttools.plt
import plttools
from plttools.plt import PLT
from plttools import plt_calculator
import matplotlib.pyplot as pl

In [3]:
# Import data from AWS S3
# AWS credentials may be provided explicitly with s3fs.S3FileSystem,
# but it is more secure to exclude the credentials from the code.
# Instead use the AWS CLI to `aws configure` credentials.

import glob
import s3fs

s3 = s3fs.S3FileSystem(anon=False)
files = s3.glob('s3://fannie-mae-phase-3/03 Data Aggregation POC/01 Test/test_4/split/*.parquet')
plts = pd.concat([pd.read_parquet('s3://' + fp) for fp in files])

In [4]:
# Import local data
plts = pd.read_parquet('sample_data/')

In [5]:
# Create PLT from imported data
json_test = plts.to_dict(orient='records')
original_plt = plttools.plt.PLT(data = json_test, number_of_simulations=50000)
original_plt.plt.head(5).style

,AccgrpId,AccountName,Lob,LobId,CedantId,LossType,PeriodId,Weight,EventId,EventDate,LossDate,Loss
0,128407,LA_128407,Unknown,0,Unknown,All,9,0.000020,2876228,2020-07-23 00:00:00,2020-07-23 00:00:00,896022.058517
1,128407,LA_128407,Unknown,0,Unknown,All,47,0.000020,2866964,2020-06-27 00:00:00,2020-06-27 00:00:00,732440.366140
2,128407,LA_128407,Unknown,0,Unknown,All,103,0.000020,2851368,2020-08-26 00:00:00,2020-08-26 00:00:00,469704.898430
3,128407,LA_128407,Unknown,0,Unknown,All,104,0.000020,2855112,2020-09-17 00:00:00,2020-09-17 00:00:00,850869.472687
4,128407,LA_128407,Unknown,0,Unknown,All,107,0.000020,2872546,2020-08-30 00:00:00,2020-08-30 00:00:00,974882.360239


In [6]:
# Group account PLTs
grouped_plt = plt_calculator.group_plts(original_plt)
grouped_plt.plt.head(5).style

,PeriodId,EventId,EventDate,LossDate,Weight,Loss
0,6,10480747,2020-01-13 00:00:00,2020-02-18 00:00:00,0.000020,137502.986024
1,9,2876228,2020-07-23 00:00:00,2020-07-23 00:00:00,0.000020,2439354.454580
2,11,2864967,2020-09-09 00:00:00,2020-09-16 00:00:00,0.000020,718498.106657
3,13,9350305,2020-10-18 00:00:00,2020-10-28 00:00:00,0.000020,150942.771282
4,26,2863292,2020-08-12 00:00:00,2020-08-12 00:00:00,0.000020,1231135.765835


In [7]:
# Rollup PLT
rolled_up_plt = plt_calculator.roll_up_plts(original_plt)
rolled_up_plt.plt.head(5)

,PeriodId,Weight,Loss
0,6,0.00002,1.375030e+05
1,9,0.00002,2.439354e+06
2,11,0.00002,7.184981e+05
3,13,0.00002,1.509428e+05
4,26,0.00002,1.231136e+06


In [8]:
# Calculate statistics from rolled up PLT
aal = rolled_up_plt.get_aal()
std = rolled_up_plt.get_standard_deviation_risk_modeler()
covvar = rolled_up_plt.get_covvar_risk_modeler()
print('AAL: ' + aal.astype(str))
print('STD: ' + str(std))
print('CovVar: ' + str(covvar))

AAL: 332927.851817096
STD: 883685.9186214344
CovVar: 2.654286548312926


In [22]:
# Calculate AEP metrics from PLT
aep = plt_calculator.calculate_aep_curve(grouped_plt.plt, grouped_plt.simulations)
aep_metrics = aep.curve['Loss'][aep.curve['Loss'].index.isin([1/x for x in aep.RETURN_PERIODS])]
aep_metrics.index = np.reciprocal(aep_metrics.index)
aep_metrics = aep_metrics.sort_index(axis=0, ascending=False)

# Return AEP results
aep_metrics.to_frame().style.format("{:,.0f}")

,Loss
Probability,
10000.0,"8,629,468"
5000.0,"7,789,031"
1000.0,"6,377,343"
500.0,"5,679,732"
250.0,"4,818,567"
200.0,"4,578,668"
100.0,"3,961,618"
50.0,"3,390,390"
25.0,"2,788,478"


In [23]:
# Calculate OEP metrics from PLT
oep = plt_calculator.calculate_oep_curve(grouped_plt.plt, grouped_plt.simulations)
oep_metrics = oep.curve['Loss'][oep.curve['Loss'].index.isin([1/x for x in oep.RETURN_PERIODS])]
oep_metrics.index = np.reciprocal(oep_metrics.index)
oep_metrics = oep_metrics.sort_index(axis=0, ascending=False)

# Return OEP results
oep_metrics.to_frame().style.format("{:,.0f}")

,Loss
Probability,
10000.0,"4,693,108"
5000.0,"4,622,154"
1000.0,"4,302,221"
500.0,"4,166,935"
250.0,"3,961,226"
200.0,"3,854,899"
100.0,"3,549,359"
50.0,"3,120,818"
25.0,"2,660,592"
